依然是pyspark版代码，似乎是为了在开发环境，无法接触到生产环境中的数据时，用于测试计算代码的逻辑

In [5]:
ben1 = get_price('000300.XSHG', '2013-01-01', '2021-11-16', 'daily', None, True, None)
ben1 = spark.createDataFrame(ben1.reset_index())
ben1 = ben1.select(
    F.substring('index', 1, 10).alias('date'),
    ((F.col('close') / F.lag('close').over(Window.orderBy('index')) - 1) * 100).alias('ben1')
    ).sort(ben1.index.desc())
ben2 = pd.DataFrame(pd.Series(ben2_ori), columns=['ben2'])
ben2 = ben2.reset_index().rename(columns= {'index': 'date'})
ben2 = spark.createDataFrame(ben2)
ben2 = ben2.select(
    F.from_unixtime(F.col('date') / 1000, 'yyyy-MM-dd').alias('date'),
    'ben2'
    ).sort(ben2.date.desc())
bench = ben1.join(ben2, 'date').withColumn(
    'benchmark',
    F.expr('(ben1 * 0.55 + ben2 * 0.45)')
    ).drop('ben1', 'ben2').sort(F.col('date').desc())
q=query(finance.FUND_NET_VALUE).filter(finance.FUND_NET_VALUE.code=="000126").order_by(finance.FUND_NET_VALUE.day.desc())
df=finance.run_query(q)
nav = spark.createDataFrame(df)
nav = nav.select(
    'code',
    nav.day.alias('date'),
    nav.net_value.alias('nav'),
    nav.sum_value.alias('acc_nav'),
    F.round(nav.refactor_net_value, 4).alias('adj_nav')
)
window = Window.partitionBy('code').orderBy('date')
nav = nav.withColumn(
    'nav_pctchg',
    F.round((F.col('nav') / F.lag('nav').over(window) - 1) * 100, 2)
    ).withColumn(
        'adj_pctchg',
        F.round((F.col('adj_nav') / F.lag('adj_nav').over(window) - 1) * 100, 2)
        )
nav = nav.sort(nav.date.desc())
nav = nav.join(bench, 'date').sort(nav.date.desc()).withColumn('riskfree', F.expr('0.007')) # 0.0069396961196233065
nav = nav.withColumn('days_per_year', F.expr('252'))
nav.show()

+----------+------+------+-------+-------+----------+----------+--------------------+--------+-------------+
|      date|  code|   nav|acc_nav|adj_nav|nav_pctchg|adj_pctchg|           benchmark|riskfree|days_per_year|
+----------+------+------+-------+-------+----------+----------+--------------------+--------+-------------+
|2021-11-16|000126|4.0861| 4.4301| 5.4918|     -1.84|     -1.84| 0.01360409793994034|   0.007|          252|
|2021-11-15|000126|4.1627| 4.5067| 5.5947|      -4.0|      -4.0|-0.04948465302339...|   0.007|          252|
|2021-11-12|000126|4.3362| 4.6802| 5.8279|       0.9|       0.9|-0.11420455436701892|   0.007|          252|
|2021-11-11|000126|4.2974| 4.6414| 5.7758|       0.1|       0.1|  0.8654815026580598|   0.007|          252|
|2021-11-10|000126|4.2931| 4.6371|   5.77|     -1.89|     -1.89| -0.2878671536331663|   0.007|          252|
|2021-11-09|000126|4.3757| 4.7197|  5.881|      0.93|      0.93|-0.02934102712771...|   0.007|          252|
|2021-11-08|000126|

In [6]:
def daily_return(nav):
    window = Window.partitionBy('code').orderBy('date')
    return nav.withColumn(
        'return',
        F.round((F.col('nav') / F.lag('nav').over(window) - 1) * 100, 4)
        ).sort(nav.date.desc())
daily_return(nav).show(5)

+----------+------+------+-------+-------+----------+----------+--------------------+--------+-------------+-------+
|      date|  code|   nav|acc_nav|adj_nav|nav_pctchg|adj_pctchg|           benchmark|riskfree|days_per_year| return|
+----------+------+------+-------+-------+----------+----------+--------------------+--------+-------------+-------+
|2021-11-16|000126|4.0861| 4.4301| 5.4918|     -1.84|     -1.84| 0.01360409793994034|   0.007|          252|-1.8402|
|2021-11-15|000126|4.1627| 4.5067| 5.5947|      -4.0|      -4.0|-0.04948465302339...|   0.007|          252|-4.0012|
|2021-11-12|000126|4.3362| 4.6802| 5.8279|       0.9|       0.9|-0.11420455436701892|   0.007|          252| 0.9029|
|2021-11-11|000126|4.2974| 4.6414| 5.7758|       0.1|       0.1|  0.8654815026580598|   0.007|          252| 0.1002|
|2021-11-10|000126|4.2931| 4.6371|   5.77|     -1.89|     -1.89| -0.2878671536331663|   0.007|          252|-1.8877|
+----------+------+------+-------+-------+----------+----------+

In [7]:
DT = '2021-11-16'   #全局常量，代表最新日期
BDT = '2020-11-17'  # 一年前的日期
def period_return(nav, bgndt=BDT, enddt=DT): #截止日期默认为最新日期
    nav = nav.filter(nav.date.between(bgndt, enddt))
    nav = daily_return(nav)
    mul_udf = F.udf(lambda x: reduce(mul, x))
    col = F.col('return') / 100 + 1
    benchcol = F.col('benchmark') / 100 + 1
    return nav.groupBy('code').agg(
        F.round((mul_udf(F.collect_list(col)) - 1) * 100, 2).alias('returnmn'),
        F.round((mul_udf(F.collect_list(benchcol)) - 1) * 100, 2).alias('bench_returnmn'),
        F.max('days_per_year').alias('days_per_year'),
        F.count('date').alias('days')).withColumn(
            'ann_returnmn',
            F.round((F.col('returnmn') / 100 + 1) ** (F.col('days_per_year') / F.col('days')) * 100 - 100, 2)
            ).withColumn(
                'active_returnmn',
                F.col('returnmn') - F.col('bench_returnmn')
                ).drop('days', 'days_per_year', 'bench_returnmn')
period_return(nav, '2019-12-31', '2020-12-31').show()

+------+--------+------------+---------------+
|  code|returnmn|ann_returnmn|active_returnmn|
+------+--------+------------+---------------+
|000126|    97.9|      102.38|          82.79|
+------+--------+------------+---------------+



In [8]:
def new_high(nav, bgndt=BDT, enddt=DT):
    top = nav.filter(nav.date <= bgndt).groupBy('code').agg(F.max('nav').alias('top_nav'))
    navs = nav.filter(nav.date.between(bgndt, enddt)).sort(nav.date).groupBy('code').agg(
        F.collect_list('nav').alias('navs')
        )
    rdds = top.join(navs, 'code').rdd.map(lambda nav: ({nav.code: [nav.navs, nav.top_nav]}))
    def _new_high(rdd):
        codes = list(rdd.keys())[0]
        values = rdd.get(codes)
        days = len(values[0])
        bgn_nav = values[0][0]
        top_nav = values[1]
        inner_break = 0
        outer_break = 0
        for i in range(0, days):
            temp = values[0][i]
            if temp > top_nav:
                outer_break += 1
                top_nav = temp
            if temp > bgn_nav:
                inner_break += 1
                bgn_nav = temp
        return codes, round(inner_break / float(days - 1) * 100, 2), round(outer_break / float(days) * 100, 2)
    return rdds.map(_new_high).toDF(['code', 'inner_nhr', 'outer_nhr'])
new_high(nav, '2020-11-16').show()

+------+---------+---------+
|  code|inner_nhr|outer_nhr|
+------+---------+---------+
|000126|    18.11|    15.98|
+------+---------+---------+



In [9]:
def profit_decline(nav, bgndt=BDT, enddt=DT):
    return nav.filter(nav.date.between(bgndt, enddt)).groupBy('code').agg(
        F.count('date').alias('total'),
        F.count(F.when(nav.nav_pctchg > 0, 'date')).alias('profit'),
        F.count(F.when(nav.nav_pctchg < 0, 'date')).alias('decline')
        ).withColumn('dec_pct', F.expr('round(decline / total * 100, 2)'))
profit_decline(nav).show()

+------+-----+------+-------+-------+
|  code|total|profit|decline|dec_pct|
+------+-----+------+-------+-------+
|000126|  243|   134|    109|  44.86|
+------+-----+------+-------+-------+



In [10]:
def roll(nav, time, date=DT):
    a = nav.withColumn(
        'date', 
        F.expr('date_add(add_months(date, -{1}), 0)'.format(date, time))
        ).select(
            'code',
            'date', 
            F.col('adj_nav').alias('roll_nav')
            )
    nav = nav.join(
        a,
        ['date', 'code'],
        'left').withColumn(
            'roll_nav',
            F.last('roll_nav', ignorenulls=True).over(Window.partitionBy('code').orderBy('date'))
            )
    nav = nav.sort(nav.date.desc()).filter('date < "2021-05-15"').withColumn(
        'roll', 
        F.expr('roll_nav / adj_nav - 1') * 100
        ).select(
            'code',
            'date',
            F.round('roll', 2).alias('roll_ret')
            )
    roll_gp = nav.groupBy('code').agg(
        F.max('roll_ret').alias('max_roll'),
        F.min('roll_ret').alias('min_roll'),
        F.round((F.count(F.when(F.col('roll_ret') > 0, 'date')) / F.count('date')) * 100, 2).alias('profit_rate')
        )
    return nav, roll_gp
roll_halfy = roll(nav, 6)
roll_halfy[1].show()
roll_halfy[0].show()

+------+--------+--------+-----------+
|  code|max_roll|min_roll|profit_rate|
+------+--------+--------+-----------+
|000126|    68.0|   -5.75|      79.42|
+------+--------+--------+-----------+

+------+----------+--------+
|  code|      date|roll_ret|
+------+----------+--------+
|000126|2021-05-14|   50.09|
|000126|2021-05-13|   53.94|
|000126|2021-05-12|    52.3|
|000126|2021-05-11|   50.99|
|000126|2021-05-10|   49.84|
|000126|2021-05-07|   58.42|
|000126|2021-05-06|   53.27|
|000126|2021-04-30|   51.61|
|000126|2021-04-29|   54.57|
|000126|2021-04-28|    50.4|
|000126|2021-04-27|   55.06|
|000126|2021-04-26|   52.89|
|000126|2021-04-23|    47.3|
|000126|2021-04-22|   47.85|
|000126|2021-04-21|   51.41|
|000126|2021-04-20|   54.39|
|000126|2021-04-19|   50.22|
|000126|2021-04-16|   50.25|
|000126|2021-04-15|   49.61|
|000126|2021-04-14|   46.46|
+------+----------+--------+
only showing top 20 rows



In [ ]:
def roll_old(nav, time, date=DT):
    last_bgndt = spark.sql(
        'select string(add_months("{0}", -{1}))'
        .format(DT, time)).first()[0] # 最新滚动期的期初日期
    diff = nav.filter(nav.date.between(last_bgndt, date)).count()  # 用最新滚动期期初期末之前的交易日数量作为偏移量
    window = Window.partitionBy('code').orderBy('date')
    roll_nav = nav.select(
        'code',
        'date',
        F.round((F.lead('adj_nav', diff).over(window) / F.col('adj_nav') - 1) * 100, 2).alias('roll_ret')
        ).dropna(subset = ['roll_ret']).sort(nav.date.desc())
    roll_gp = roll_nav.groupBy('code').agg(
        F.max('roll_ret').alias('max_roll'),
        F.min('roll_ret').alias('min_roll'),
        F.round((F.count(F.when(F.col('roll_ret') > 0, 'date')) / F.count('date')) * 100, 2).alias('profit_rate')
        )
    return roll_nav, roll_gp
roll_halfy = roll(nav, 6)
roll_halfy[1].show()
roll_halfy[0].show()

In [11]:
def vol(nav, bgndt=BDT, enddt=DT):
    return nav.filter(nav.date.between(bgndt, enddt)).groupBy('code').agg(
        F.round(F.stddev_pop('nav_pctchg'), 4).alias('vol'),
        F.max('days_per_year').alias('days_per_year')
        ).withColumn(
            'ann_vol',
            F.expr('round(vol * sqrt(days_per_year), 4)')
            ).drop('days_per_year')
vol(nav).show()

+------+------+-------+
|  code|   vol|ann_vol|
+------+------+-------+
|000126|2.0978|33.3015|
+------+------+-------+



In [12]:
def sortino_sharpe(nav, bgndt=BDT, enddt=DT):
    adj_risk = F.col('nav_pctchg') - F.col('riskfree')
    adj_risk0 = F.when(adj_risk > 0 , 0).otherwise(adj_risk)
    nav = nav.filter(nav.date.between(bgndt, enddt)).groupBy('code').agg(
        F.sum(adj_risk0 ** 2).alias('sum'),
        F.mean(adj_risk).alias('mean'),
        F.stddev(adj_risk).alias('std'),
        F.count('date').alias('cnt'),
        F.max('days_per_year').alias('dpy')
        )
    return nav.withColumn(
            'down_vol',
            F.expr('sqrt(dpy * sum / (cnt - 1))')
            ).withColumn(
                'sortino',
                F.expr('dpy * mean / down_vol')
                ).withColumn(
                    'sharpe',
                    F.expr('sqrt(dpy) * mean / std')
                ).select(
                    'code',
                    F.round('down_vol', 2).alias('down_vol'),
                    F.round('sortino', 4).alias('sortino'),
                    F.round('sharpe', 4).alias('sharpe')
                    )
sortino_sharpe(nav).show()

+------+--------+-------+------+
|  code|down_vol|sortino|sharpe|
+------+--------+-------+------+
|000126|   21.45| 2.6919|1.7306|
+------+--------+-------+------+



In [13]:
def track(nav, bgndt=BDT, enddt=DT):
    active = (F.col('nav_pctchg') - F.col('benchmark')) / 100
    nav = nav.filter(nav.date.between(bgndt, enddt)).groupBy('code').agg(
        F.stddev_pop(active).alias('std'),
        F.mean(active).alias('mean'),
        F.max('days_per_year').alias('dpy')
        )
    return nav.withColumn(
        'track_error',
        F.expr('std * sqrt(dpy)')
        ).withColumn(
            'info',
            F.expr('mean / track_error * dpy')
            ).select(
                'code',
                F.round('track_error', 4).alias('track_error'),
                F.round('info', 4).alias('info')
                )
track(nav, '2019-12-31', '2020-12-31').show()

+------+-----------+-----+
|  code|track_error| info|
+------+-----------+-----+
|000126|     0.2307|2.626|
+------+-----------+-----+



In [14]:
def drawdown(nav):
    # 回撤曲线
    window = Window.partitionBy('code').orderBy('date')
    return nav.withColumn(
        'drawdown',
        F.round((1 - F.col('adj_nav') / F.max('adj_nav').over(window)) * 100, 4)
        ).sort(nav.date.desc())
nav_dd = drawdown(nav)
nav_dd.show()

+----------+------+------+-------+-------+----------+----------+--------------------+--------+-------------+--------+
|      date|  code|   nav|acc_nav|adj_nav|nav_pctchg|adj_pctchg|           benchmark|riskfree|days_per_year|drawdown|
+----------+------+------+-------+-------+----------+----------+--------------------+--------+-------------+--------+
|2021-11-16|000126|4.0861| 4.4301| 5.4918|     -1.84|     -1.84| 0.01360409793994034|   0.007|          252|  9.5687|
|2021-11-15|000126|4.1627| 4.5067| 5.5947|      -4.0|      -4.0|-0.04948465302339...|   0.007|          252|  7.8743|
|2021-11-12|000126|4.3362| 4.6802| 5.8279|       0.9|       0.9|-0.11420455436701892|   0.007|          252|  4.0343|
|2021-11-11|000126|4.2974| 4.6414| 5.7758|       0.1|       0.1|  0.8654815026580598|   0.007|          252|  4.8922|
|2021-11-10|000126|4.2931| 4.6371|   5.77|     -1.89|     -1.89| -0.2878671536331663|   0.007|          252|  4.9877|
|2021-11-09|000126|4.3757| 4.7197|  5.881|      0.93|   

In [15]:
def max_drawdown(nav_dd, bgndt=BDT, enddt=DT):
    return nav_dd.filter(nav_dd.date.between(bgndt, enddt)).groupBy('code').agg(
        F.max('drawdown').alias('drawdown')
        )
max_drawdown(nav_dd).show()

+------+--------+
|  code|drawdown|
+------+--------+
|000126| 19.9792|
+------+--------+



In [16]:
def drawdown_recover(nav_dd, bgndt=BDT, enddt=DT):
    max_dt = nav_dd.join(
        max_drawdown(nav_dd, bgndt, enddt),
        ['code', 'drawdown']).select(
            'code',
            F.col('adj_nav').alias('max_adj_nav'), 
            F.col('date').alias('max_dt'),
            F.col('drawdown').alias('max_drawdown')
            )
    rec_dt = nav_dd.join(
        max_dt,
        'code'
        ).filter(nav_dd.date > max_dt.max_dt).groupBy('code').agg(
           F.min(F.when(
                F.expr('drawdown = 0'), F.col('date'))).alias('rec_dt')
            )
    return nav_dd.join(
        max_dt,
        'code'
        ).join(
            rec_dt,
            'code'
            ).filter(nav.date.between(max_dt.max_dt, rec_dt.rec_dt)).groupBy('code').count()
drawdown_recover(nav_dd).show()

+------+-----+
|  code|count|
+------+-----+
|000126|   55|
+------+-----+



In [17]:
def ac_drawdown(nav):
    # 主动收益回撤曲线
    window = Window.partitionBy('code').orderBy('date')
    nav = nav.withColumn(
        'active_nav',
        F.expr('nav / (100 + nav_pctchg) * (100 + nav_pctchg - benchmark)')
        )
    return nav.withColumn(
        'drawdown',
        F.round((1 - F.col('active_nav') / F.max('active_nav').over(window)) * 100, 4)
        ).sort(nav.date.desc())
max_drawdown(ac_drawdown(nav)).show()

+------+--------+
|  code|drawdown|
+------+--------+
|000126| 18.0748|
+------+--------+



In [18]:
def max_decline(nav, bgndt=BDT, enddt=DT):
    return nav.filter(nav.date.between(bgndt, enddt)).groupBy('code').agg(
        F.min('nav_pctchg').alias('max_decline')
        )
max_decline(nav).show()

+------+-----------+
|  code|max_decline|
+------+-----------+
|000126|      -4.96|
+------+-----------+



In [19]:
def keep_decline(nav, bgndt=BDT, enddt=DT):
    window = Window.partitionBy('code').orderBy('date')
    nav = nav.select(
        'code',
        'date',
        'nav_pctchg',
        F.row_number().over(window).alias('rn'),
        F.lag('nav_pctchg').over(window).alias('ret_lag')
        ).withColumn(
            'decline',
            F.sum(F.expr('if(nav_pctchg < 0 and ret_lag < 0, 1, 0)')).over(window)
            ).withColumn('diff', F.expr('rn - decline'))
    return nav.groupBy('code', 'diff').agg(
        F.count('diff').alias('keep_decline')
        ).groupBy('code').agg(F.max('keep_decline').alias('max_keep_decline'))
keep_decline(nav).show()

+------+----------------+
|  code|max_keep_decline|
+------+----------------+
|000126|               7|
+------+----------------+



In [20]:
def calmar(nav, bgndt=BDT, enddt=DT):
    window = Window.partitionBy('code')
    mean = nav.filter(nav.date.between(bgndt, enddt)).groupBy('code').agg(
        F.mean('nav_pctchg').alias('mean1'),
        F.mean('riskfree').alias('mean2')
    )
    dd = max_drawdown(drawdown(nav), bgndt, enddt)
    return mean.join(dd, 'code').select(
        'code',
        F.expr('(mean1 - mean2) / drawdown * 100').alias('calmar')
        )
calmar(nav).show()

+------+------------------+
|  code|            calmar|
+------+------------------+
|000126|1.1470571369532954|
+------+------------------+



In [21]:
def beta_alpha(nav, bgndt=BDT, enddt=DT):
    nav = nav.filter(nav.date.between(bgndt, enddt))
    rdds = nav.groupBy('code').agg(
        F.collect_list('nav_pctchg').alias('returnmn'),
        F.collect_list('riskfree').alias('riskfreemn'),
        F.collect_list('benchmark').alias('benchmarkmn')
        ).rdd.map(lambda nav : ({nav.code: [nav.returnmn, nav.riskfreemn, nav.benchmarkmn]}))
    def beta(rdd):
        codes = list(rdd.keys())[0]
        values = rdd.get(codes)
        ret = np.array(values[0]).astype(float)
        risk = np.array(values[1]).astype(float)
        bench = np.array(values[2]).astype(float)
        cov = np.cov(ret - risk, bench - risk)
        beta = cov[0][1] / cov[1][1]
        return codes, np.float(beta)
    df = rdds.map(beta).toDF(['code', 'beta'])
    nav = nav.join(df, 'code')
    col = F.expr('nav_pctchg - riskfree - beta * (benchmark - riskfree)')
    return nav.groupBy('code').agg(
        F.round(F.max('beta'), 4).alias('beta'),
        F.round(F.mean(col), 4).alias('alpha')
    )
beta_alpha(nav, '2020-11-16').show()

+------+------+------+
|  code|  beta| alpha|
+------+------+------+
|000126|1.8872|0.2218|
+------+------+------+

